In [0]:
%pip install databricks-feature-engineering==0.6 databricks-sdk==0.32.0
%restart_python

In [0]:
import os

from hotel_reservation.config import ProjectConfig
from hotel_reservation.serving.fe_model_serving import FeatureLookupServing
from hotel_reservation.utils import configure_logging

logger = configure_logging("Hotel Reservations Deploy Model")

# Load project config
config = ProjectConfig.from_yaml(config_path="../project_config.yml")
logger.info("Configuration loaded")

# Initialize feature store manager
feature_serving_manager = FeatureLookupServing(model_name=f'{config.catalog_name}.{config.schema_name}.hotel_reservation_model_fe', endpoint_name='hotel_reservation_endpoint_koen', feature_table_name=f'{config.catalog_name}.{config.schema_name}.hotel_reservation_features')


In [0]:

# Create online table
feature_serving_manager.create_online_table()

# Deploy feature serving endpoint
feature_serving_manager.deploy_or_update_serving_endpoint()
logger.info("Created feature serving endpoint.")

In [0]:
import mlflow
from mlflow.models import Model

model_uri = 'runs:/621e8115ff434d3da24d6420574c44eb/HistGradientBoostingClassifier-model-fe'
# The model is logged with an input example
pyfunc_model = mlflow.pyfunc.load_model(model_uri)

In [0]:
# Create a sample request body
required_columns = ['no_of_weekend_nights', 'no_of_week_nights', 'required_car_parking_space', 'lead_time', 'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled', 'no_of_special_requests', 'Booking_ID', 'market_segment_type_Complementary', 'market_segment_type_Corporate', 'market_segment_type_Offline', 'market_segment_type_Online', 'type_of_meal_plan_Meal_Plan_2', 'room_type_reserved_Room_Type_4', 'room_type_reserved_Room_Type_6', 'room_type_reserved_Room_Type_7', 'quarter_Q2', 'quarter_Q3', 'quarter_Q4', 'no_of_adults', 'no_of_children', 'avg_price_per_room', 'cancellation_probability']

train_set = spark.table(f"{config.catalog_name}.{config.schema_name}.train_dataset").toPandas()
train_set['cancellation_probability'] = 0
sampled_records = train_set[required_columns].sample(n=1000, replace=True).to_dict(orient="records")
dataframe_records = [[record] for record in sampled_records]

# Call the endpoint with one sample record
os.environ["DBR_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
os.environ["DBR_HOST"] = spark.conf.get("spark.databricks.workspaceUrl")

status_code, response_text = feature_serving_manager.call_endpoint([sampled_records[0]])
logger.info(f"Response Status: {status_code}")
logger.info(f"Response Text: {response_text}")

In [0]:
model_uri = 'runs:/621e8115ff434d3da24d6420574c44eb/HistGradientBoostingClassifier-model-fe'
# The model is logged with an input example
pyfunc_model = mlflow.pyfunc.load_model(model_uri)

pyfunc_model.metadata.signature.inputs.input_names()